# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.common import ML_api
from Clust.clust.ML.tool import meta

influxdb_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

cpu is available.


# 1. set param from Front End

In [2]:
# feature list와 data_y_flag는 model_info에서 받아와야 합니다.
# regression
params1 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testX',
        "feature_list": "None"
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testy',
        "feature_list": "None"
    },
    'data_y_flag' : "None",
    'model_name':'regression_energy_cleanLevel4_trainX_regression_GRU_rg_'
}

# forecasting
params2 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list": "None"
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list": "None"
    },
    'data_y_flag' : "None",
    'model_name':'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX_regression_GRU_rg_'
}

# classification
params3 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testX',
        "feature_list": "None"
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testy',
        "feature_list": "None"
    },
    'data_y_flag' : "None",
    'model_name':'classification_actionPattern_cleanLevel0_trainX_classification_GRU_cf_'
}

# 2. Backend

In [3]:
# 1. Backend Parameter Setting
params = params1

# chage tpye string to bool -> ex) 'true' -> True
params = ML_api.chagne_type_str_to_bool(params)

In [4]:
# 2. get model meta
model_name = params['model_name']
meta_file_name = "./meta.json"
# for testing mongodb model
# model_meta = meta.read_model_meta_from_mongodb(mongodb_client, 'model', 'meta', model_name)
# for testing local model
model_meta = meta.read_model_meta_from_local(meta_file_name, model_name)

# data parameters from model meta
params['ingestion_param_X']['feature_list'] = model_meta['ingestion_param_X']['feature_list']
params['ingestion_param_y']['feature_list'] = model_meta['ingestion_param_y']['feature_list']
params['data_y_flag'] = model_meta['data_y_flag']

# scaler/transform info for test data preparation
params['scaler_param'] = model_meta['scaler_param']
params['transform_param'] = model_meta['transform_param']

# model info for ML test
params['model_info'] = model_meta['model_info']

In [5]:
# 3. Data Preparation
# make test data, scaler
test_X_array, test_y_array, scaler_X, scaler_y = ML_api.test_data_preparation(params, influxdb_client)

{'bucket_name': 'integration', 'ms_name': 'regression_energy_cleanLevel4_testX', 'feature_list': ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']}


window_size: 144 nan_limit_num: 72
(6048, 24) (42, 144, 24)
(42, 1) (42, 1)


In [6]:
# set scaler by data y flag
if params['data_y_flag']:
    scaler = scaler_y
else:
    scaler = scaler_X

In [7]:
# model test
result = ML_api.ML_test(params, test_X_array, test_y_array, scaler)

features shape: torch.Size([42, 144, 24]) targets shape:  torch.Size([42, 1])

Start testing data

scale
=========================data
        value  prediction
0   17.370001    7.650870
1   20.650000    7.562680
2   11.420000    7.149018
3   10.680000    7.541431
4   12.439999    7.565461
5   11.170000    7.380070
6   24.120001    7.650904
7   10.990000    7.690277
8   13.760000    7.368565
9   14.560000    7.388246
10  14.970000    7.460073
11  13.429999    7.464605
12  11.570000    7.626628
13   9.330000    7.931899
14  15.580000    7.759978
15  20.929998    7.646781
16  11.990000    7.403240
17  15.370000    7.682791
18  10.470000    7.525137
19  14.410000    7.600495
20  10.160000    7.498172
21  15.120000    7.460516
22  12.320000    7.555345
23  10.460000    7.281071
24  15.060000    6.951315
25   9.680000    7.198527
26  10.690000    7.611364
27  17.799997    7.626010
28  10.690000    7.720323
29  17.059998    7.561688
30  20.879999    7.566436
31  10.890000    7.503093


In [8]:
result['result_metrics']

{'mae': 6.4199057,
 'rmse': 7.4158515339365225,
 'r2': -2.9340607269929446,
 'mape': 42.62121021747589}

In [9]:
model_meta['model_info']

{'model_purpose': 'regression',
 'model_method': 'GRU_rg',
 'model_name': 'regression_energy_cleanLevel4_trainX_regression_GRU_rg_',
 'model_tags': 'tagstest',
 'train_parameter': {'lr': 0.0001,
  'weight_decay': 1e-06,
  'n_epochs': 5,
  'batch_size': 16,
  'device': 'cpu'},
 'model_parameter': {'rnn_type': 'lstm',
  'input_size': 24,
  'hidden_size': 64,
  'num_layers': 2,
  'output_dim': 1,
  'seq_len': 144,
  'dropout': 0.1,
  'bidirectional': True},
 'seq_len': 144,
 'input_size': 24,
 'model_file_path': {'modelFile': {'fileName': 'model.pth',
   'filePath': './Models/GRU_rg/regression_energy_cleanLevel4_trainX_regression_GRU_rg_/regression_energy_cleanLevel4_trainX/model.pkl'}}}